# Script de predicción 

### Índice   
1. [Carga de datos](#id1)

    1.1 [Valores nulos en los datos](#id11)
    
    1.2 [Las columnas READINGTHOUSANTH y DELTATHOUSANDTH](#id12)
    
    1.3 [La columna SAMPLETIME](#id13)
    
   
2. [Procesado de datos: las columnas DELTA y READING](#id2)

    2.1 [Agregar los consumos por fechas](#id21)
    
    2.2 [Completar medidas faltantes](#id22)
    
    2.3 [Tipos de contadores según su serie temporal de consumos](#id23)
    
    2.4 [Tratar outliers en los consumos y medidas negativas](#id24)
    
    
3. [Predicciones contadores tipos 0, 1 y 3](#id3)

    3.1 [Tipo 0](#id31)
    
    3.2 [Tipo 1](#id32)
    
    3.3 [Tipo 3](#id33)
    
    
4. [Predicciones contadores tipo 2](#id4)

    4.1 [AutoARIMA](#id41)
    
    4.2 [XGBoost](#id42)
    
    
5. [Postprocesado](#id5)

In [168]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import datetime
import math

from pmdarima.arima import auto_arima
import json
import xgboost as xgb
import warnings

In [169]:
warnings.simplefilter('ignore')

## Carga de datos<a name="id1"></a>

Cargamos los datos y observamos que tenemos

In [4]:
df = pd.read_csv('../data/Modelar_UH2022.txt',sep = '|') 
df.head()

,ID,SAMPLETIME,READINGINTEGER,READINGTHOUSANDTH,DELTAINTEGER,DELTATHOUSANDTH
0,0,2019-06-13 08:34:09,369320,0.0,17,0.0
1,0,2019-06-13 17:34:10,369403,0.0,2,0.0
2,0,2019-06-13 18:34:10,369403,0.0,0,0.0
3,0,2019-06-13 04:34:10,369284,0.0,1,0.0
4,0,2019-06-13 14:34:10,369356,0.0,28,0.0


Vemos el tamaño de los datos:

In [5]:
df.shape

(21404828, 6)

### Valores nulos en los datos<a name="id11"></a>

Vemos si hay valores nulos en los datos:

In [6]:
df.isna().sum()

ID                        0
SAMPLETIME                0
READINGINTEGER            0
READINGTHOUSANDTH    140056
DELTAINTEGER              0
DELTATHOUSANDTH      140056
dtype: int64

Como los valores nulos pertenecen a la parte de las milésimas los despreciamos y los imputamos a 0:

In [7]:
df = df.fillna(0)

### Las columnas READINGTHOUSANDTH y DELTATHOUSANDTH<a name="id12"></a>

Examinemos las columnas *READINGTHOUSANDTH* y *DELTATHOUSANDTH*:

In [15]:
print('Number READINGTHOUSANTH != 0:',(df['READINGTHOUSANDTH']!=0).sum())
print('Number DELTATHOUSANTH != 0:',(df['READINGTHOUSANDTH']!=0).sum())
print('Minimum READINGTHOUSANTH:',df['READINGTHOUSANDTH'].min())
print('Minimum DELTATHOUSANTH:',df['DELTATHOUSANDTH'].min())
print('Maximum READINGTHOUSANTH:',df['READINGTHOUSANDTH'].max())
print('Maximum DELTATHOUSANTH:',df['DELTATHOUSANDTH'].max())

Number READINGTHOUSANTH != 0: 2428718
Number DELTATHOUSANTH != 0: 2428718
Minimum READINGTHOUSANTH: 0.0
Minimum DELTATHOUSANTH: 0.0
Maximum READINGTHOUSANTH: 99.0
Maximum DELTATHOUSANTH: 99.0


Aunque el valor máximo para estas columnas es 99, el nombre 'Thousandth' deja claro que estas columnas hacen referencia a las milésimas por lo que dividiremos su valor entre 1000 y se lo agregaremos a la parte de las unidades. No obstante, no creemos que estas dos columnas vayan a tener un gran impacto en las predicciones ya que como máximo estaremos añadiendo 0,099 litros a la medición.

Tras agregarlo a las columnas de las unidades nos desharemos tanto de las columnas de las unidades como de las de las milésimas, quedándonos con el agregado de ambas al cual llamaremos *READING* y *DELTA* a partir de ahora:

In [21]:
df['DELTA'] = [j+(i/1000) for (j,i) in zip(df['DELTAINTEGER'].values, df['DELTATHOUSANDTH'].values)]
df['READING'] = [j+(i/1000) for (j,i) in zip(df['READINGINTEGER'].values, df['READINGTHOUSANDTH'].values)]
df.drop(['READINGINTEGER','READINGTHOUSANDTH','DELTAINTEGER','DELTATHOUSANDTH'], axis=1, inplace=True)
df.head()

,ID,SAMPLETIME,DELTA,READING
0,0,2019-06-13 08:34:09,17.0,369320.0
1,0,2019-06-13 17:34:10,2.0,369403.0
2,0,2019-06-13 18:34:10,0.0,369403.0
3,0,2019-06-13 04:34:10,1.0,369284.0
4,0,2019-06-13 14:34:10,28.0,369356.0


### La columna SAMPLETIME<a name="id13"></a>

La columna *SAMPLETIME* hace referencia al día y la hora a la que las medidas *READING* y *DELTA* fueron tomadas. Para poder trabajar comodamente con esta columna vamos a convertirla de tipo *string* a tipo *datetime* y a extraer de ella una nueva columna *DATE* en la que solo tendremos la fecha de la medida (sin la hora).

In [25]:
def get_fecha(date):
    return date.strftime("%Y-%m-%d")

def str2date(string):
    return datetime.datetime.strptime(string, '%Y-%m-%d %H:%M:%S')

In [26]:
tqdm.pandas()
df['SAMPLETIME'] = df['SAMPLETIME'].progress_apply(str2date)
df['DATE'] = df['SAMPLETIME'].progress_apply(get_fecha)
df.head()

100%|███████████████████████████████████████████████████████████████████| 21404828/21404828 [05:06<00:00, 69813.81it/s]


,ID,SAMPLETIME,DELTA,READING,DATE
0,0,2019-06-13 08:34:09,17.0,369320.0,2019-06-13
1,0,2019-06-13 17:34:10,2.0,369403.0,2019-06-13
2,0,2019-06-13 18:34:10,0.0,369403.0,2019-06-13
3,0,2019-06-13 04:34:10,1.0,369284.0,2019-06-13
4,0,2019-06-13 14:34:10,28.0,369356.0,2019-06-13


Antes de continuar ordenamos el dataframe por *ID* y *SAMPLETIME* para que sea más interpretable que si está desordenado:

In [27]:
df = df.sort_values(['ID','SAMPLETIME']).reset_index(drop=True)

## Procesado de datos: las columnas DELTA y READING<a name="id2"></a>

Antes de empezar con el tratamiento de las columnas DELTA y READING hay que resaltar que la mayoría de las decisiones que vamos a tomar en este apartado están justificadas por las conclusiones obtenidas en el script de exploración por lo que no nos detendremos a explicar como hemos llegado a ellas (para ello mejor ver el script de exploración).

En primer lugar vamos a definir una función que dadas dos fechas nos devuelva una lista de todas las fechas intermedias. Está función será una utilidad que emplearemos múltiples veces a lo largo de este notebook.

In [12]:
'''
given a start date in datetime format "start_date" and an "end_date" returns a list of strings with the dates from
"start_date" to "end_date".

Example:

start_date = datetime.date(2019, 9 , 30)
end_date = datetime.date(2019, 10, 7)
get_date_range(start_date, end_date)
'''
def get_date_range(start_date, end_date):
    number_of_days = (end_date-start_date).days
    return [(start_date + datetime.timedelta(days = day)).isoformat() for day in range(number_of_days+1)]

### Agregar los consumos por fechas<a name="id21"></a>

En este punto el primer paso claro a dar es agregar por fecha los consumos. No obstante nos encontramos con un serio problema: faltan muchas medidas. Para solventar esto vamos a hacer uso de las dos columnas *DELTA* y *READING*. En un principio si todos los datos están correctos consideramos que *READING* es más fiable. De no ser así, recurrimos a *DELTA* para tratar de completar los datos. Seguiremos el siguiente algoritmo:

- IF no hay ninguna medida para un día:
    * De momento lo dejamos como **NONE** y lo completaremos más adelante
- IF hay 24 medidas, es decir, está completo:
    * IF hay 24 medidas para el día anterior:
        + Tomamos $max(READING_{actual})-max(READING_{anterior})$ 
    * IF no hay 24 medidas para el día anterior, es decir, está incompleto:
        + Tomamos $sum(DELTA_{actual})$
- IF no hay 24 medidas, es decir, está incompleto
    * Tomamos $24/N_{medidas}*sum(DELTA_{actual})$, es decir, calculamos el promedio de las medidas que haya y lo múltiplicamos por 24 (como si hubiese 24 medidas).

In [32]:
start_date = datetime.date(2019, 2 , 1)
end_date = datetime.date(2020, 1, 31)
complete_year = get_date_range(start_date, end_date)

delta_df = pd.DataFrame(columns=df['ID'].unique(), index =complete_year)

#primero rellenamos la primera columna
date = complete_year[0]
for i in tqdm(df['ID'].unique()):
    one_counter = df[df['ID']==i]
    # si no hay ninguna medida
    if len(one_counter[one_counter['DATE']==date]) == 0:
        delta_df[i][date] = None
    # si el contador está completo para ese dia
    elif len(one_counter[one_counter['DATE']==date]) >= 24:
        delta_df[i][date] = one_counter[one_counter['DATE']==date]['DELTA'].sum()
    # si el contador no está completo para ese dia
    else:
        delta_df[i][date] = (24/len(one_counter[one_counter['DATE']==date])) * \
                         one_counter[one_counter['DATE']==date]['DELTA'].sum()

100%|██████████████████████████████████████████████████████████████████████████████| 2747/2747 [02:17<00:00, 19.91it/s]


In [33]:
for i in tqdm(df['ID'].unique()):
    one_counter = df[df['ID']==i]
    for j in range(1, len(complete_year)):
        date = complete_year[j]
        # si no hay ninguna medida
        if len(one_counter[one_counter['DATE']==date]) == 0:
            delta_df[i][date] = None
        # si el contador está completo para ese dia
        elif len(one_counter[one_counter['DATE']==date]) >= 24:
            # si el contador está completo para el dia anterior
            if len(one_counter[one_counter['DATE']==complete_year[j-1]]) >= 24:
                delta_df[i][date] = one_counter[one_counter['DATE']==complete_year[j]]['READING'].max() - \
                                    one_counter[one_counter['DATE']==complete_year[j-1]]['READING'].max()
            # si el contador no está completo para el dia anterior
            else:
                delta_df[i][date] = one_counter[one_counter['DATE']==date]['DELTA'].sum()
        # si el contador no está completo para ese dia
        else:
            delta_df[i][date] = (24/len(one_counter[one_counter['DATE']==date])) * \
                             one_counter[one_counter['DATE']==date]['DELTA'].sum()

100%|████████████████████████████████████████████████████████████████████████████| 2747/2747 [2:07:09<00:00,  2.78s/it]


Tras aplicar este algoritmo nos quedamos con un dataframe en el que hemos completado todos los días para los que había al menos 1 medida. Sin embargo, los días para los que no había ninguna medida siguen siendo **NONE**. Por lo que necesitaremos otras estrategias para completarlos.

### Completar medidas faltantes<a name="id22"></a>

Empezaremos por completar los **NONE** comprendidos entre dos días para los cuales sí se tienen medidas, es decir, que no están ni al principio ni al final de la serie temporal. Para ello tomaremos la última medida *READING* del día en el que empieza la secuencia de **NONE** y la primera medida *READING* del día que termina la secuencia. Si restamos estas dos medidas obtendremos el consumo de agua total durante los días sin medidas, es decir, durante la secuencia de **NONE**. Para interpolar entre estas dos medidas dividiremos este consumo total entre el número de días sin medidas.

In [112]:
def complete_huecos(deltas, readings):
    consecutive_nones = 0
    last_date_not_none = None
    none_dates = []
    for date in deltas.index:
        if deltas[date] == None:
            consecutive_nones += 1
            none_dates.append(date)
        elif deltas[date] != None:
            if consecutive_nones > 0:
                begin = readings[readings['DATE']==last_date_not_none]['READING'].max()
                end = readings[readings['DATE']==date]['READING'].min()
                for date_2 in none_dates:
                    deltas[date_2] = (end-begin)/consecutive_nones
            consecutive_nones = 0
            last_date_not_none = date  
            none_dates = []
    return deltas

In [113]:
for i in tqdm(df['ID'].unique()): 
    delta_df[i] = complete_huecos(delta_df[i], df[df['ID']==i][['DATE', 'READING']])

100%|██████████████████████████████████████████████████████████████████████████████| 2747/2747 [17:44<00:00,  2.58it/s]


### Tipos de contadores según su serie temporal de consumos<a name="id23"></a>

Tras este procesado tenemos tres tipos de contadores:
- Contadores completos
- Contadores a los que les faltan medidas al principio
- Contadores a los que les faltan medidas al final
- Contadores a los que les faltan medidas al principo y al final

Vamos a dividir de manera más formal los contadores en tipos según su serie temporal de consumos:
- **Tipo 1:** Contadores cuyas medidas sean todas 0.
- **Tipo 2:** Contadores completos al menos en Enero de 2020 no pertenecientes al Tipo 1.
- **Tipo 3:** Contadores sin medidas en Noviembre, Diciembre de 2019 y Enero de 2020 pero con medidas en Febrero de 2019 no pertenecientes al Tipo 1.
- **Tipo 0:** Contadores no pertencientes a ninguno de los anteriores tipos.

Esta división es muy importante ya que dependiendo del tipo del contador se le aplicará una estrategia de predicción u otra. A continuación vamos a fabricar un diccionario en el cual las claves sean los identificadores de los 4 tipos y los valores listas con los IDs pertenecientes a ese tipo:

In [15]:
dicc_tipos = {'tipo_1':[], 'tipo_2':[], 'tipo_3':[], 'tipo_0':[]}

for i in tqdm(delta_df.columns):
    if delta_df[i].sum()==0:
        dicc_tipos['tipo_1'].append(i)
    elif delta_df[i][delta_df[i].index >= '2020-01-01'].isna().sum()==0:
        dicc_tipos['tipo_2'].append(i)
    elif delta_df[i][delta_df[i].index <= '2019-02-14'].isna().sum()==0:
        dicc_tipos['tipo_3'].append(i)
    else:
        dicc_tipos['tipo_0'].append(i)
        
for t in dicc_tipos:
    print(t+':', len(dicc_tipos[t]))

100%|████████████████████████████████████████████████████████████████████████████| 2747/2747 [00:00<00:00, 3102.93it/s]

tipo_1: 69
tipo_2: 2534
tipo_3: 44
tipo_0: 100


### Tratar outliers en los consumos y medidas negativas<a name="id24"></a>

El propósito de esta sección es tratar los outliers en las series temporales de cada contador. Para ello recorrermos todos los contadores en busca de outliers. Consideraremos un outlier cualquier medida *DELTA* tal que

$DELTA > Q_3 + IQR \\ ó  \\ DELTA < Q_1 - IQR$

donde $Q_1$ y $Q_3$ son el primer y tercer cuartil respectivamente y $IQR$ es el rango intercuartílico. Transformaremos estos outliers igualándolos a la media de consumos del contador.

In [6]:
delta_df = pd.read_pickle('../data/counters_in_rows_3.pkl')

In [7]:
delta_df.fillna(value=np.nan, inplace=True)

In [8]:
def replace_outliers(delta_serie, IQR, Q1, Q3, avg):
    values = delta_serie.values
    out = []
    for delta in values:
        if not np.isnan(delta) and (delta < Q1 - 1.5*IQR or delta > Q3 + 1.5*IQR):
            out.append(avg)
        else:
            out.append(delta)
    return out

In [9]:
for i in tqdm(delta_df.columns):
    Q1 = np.percentile(delta_df[i][delta_df[i].notna()], 25, method = 'midpoint')
    Q3 = np.percentile(delta_df[i][delta_df[i].notna()], 75, method = 'midpoint')
    IQR = Q3 - Q1
    avg = np.mean(delta_df[i][delta_df[i].notna()])
    delta_df[i] = replace_outliers(delta_df[i], IQR, Q1, Q3, avg)

100%|█████████████████████████████████████████████████████████████████████████████| 2747/2747 [00:04<00:00, 571.64it/s]


Por otro lado, no tiene ningún sentido que un dato de consumo sea negativo. Si los datos fuesen perfectos esto no pasaría, pero no es el caso y a pesar de los tratamientos que hemos hecho hasta ahora sigue habiendo algunos consumos negativos. Sencillamente transformaremos estos consumos en ceros.

In [10]:
delta_df[delta_df < 0] = 0

## Predicciones contadores tipos 0, 1 y 3<a name="id3"></a>

El propósito de esta sección es hacer predicciones de consumo para las dos primeras semanas de Febrero de 2020 para cada contador de los tipos 0, 1 y 3. Según el tipo del contador emplearemos estrategias más o menos sofisiticadas. Recordemos cuantos contadores hay de cada tipo:

- **Tipo 1:** 68
- **Tipo 2:** 2535
- **Tipo 3:** 44
- **Tipo 0:** 100

Nuestros mayores esfuerzos estarán volcados en los contadores de **tipo 2** que tendrán una sección propia, ya que la inmensa mayoría de contadores son de este tipo y además son los más completos. Para ir recopilando las predicciones nos ayudaremos de un DataFrame *results_df* que iremos completando.

In [13]:
start_date = datetime.date(2020, 2 , 1)
end_date = datetime.date(2020, 2, 14)
prediction_dates = get_date_range(start_date, end_date)

results_df = pd.DataFrame(columns = delta_df.columns, index = prediction_dates)

### Tipo 0<a name="id31"></a>

Los contadores de **tipo 0** o bien no tienen medidas en Enero de 2020 ni en Febrero de 2019, o bien tienen muy pocas. En cualquier caso son contadores muy difíciles de predecir ya que no tenemos la cantidad apropiada de datos y en el caso de que sí la tengamos son datos poco relevantes (como los consumos de Junio). 

En este caso hemos considerado que la mejor opción es hacer una estimación grosera. Para el contador $i$ y fecha $j$ se predecirá:

$PREDICTION_{i, j}=avg(DELTA_{i})$

donde $avg(DELTA_{i})$ es la media de consumos del contador $i$. A pesar de que la media puede no ser precisa ya que está fabricada a partir de pocos datos o datos lejanos a Febrero de 2020, creemos que es la mejor estimación posible en este caso.

In [16]:
start_date = datetime.date(2019, 2 , 1)
end_date = datetime.date(2019, 2, 14)
february_2019_two_weeks = get_date_range(start_date, end_date)

for i in tqdm(dicc_tipos['tipo_0']):
    predictions = []
    avg_delta = np.mean(delta_df[i][delta_df[i].notna()])
    for j in range(14):
        predictions.append(avg_delta)
    results_df[i] = predictions

100%|██████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 1538.64it/s]


### Tipo 1<a name="id32"></a>

Los contadores de **tipo 1** son contadores cuyas medidas han sido todas 0 (sean muchas o pocas). Probablemente sean contadores ubicados en viviendas deshabitadas en las que no se consume agua por lo que hemos considerado que la mejor opción es predecir consumo 0 para estos contadores. 

$PREDICTION_{i, j}=0$

In [17]:
for i in tqdm(dicc_tipos['tipo_1']):
    predictions = []
    for j in range(14):
        predictions.append(0)
    results_df[i] = predictions

100%|████████████████████████████████████████████████████████████████████████████████| 69/69 [00:00<00:00, 2997.98it/s]


### Tipo 3<a name="id33"></a>

Los contadores de **tipo 3** son aquellos sin suficientes medidas en Enero de 2020 (el mes previo a la predicción) pero con medidas en Febrero de 2019 (el mes de la predicción pero un año antes). Para estos contadores consideramos que una mejor estimación que sencillamente la media es la siguiente:

$PREDICTION_{i, j}=\frac{1}{2}\cdot DELTA_{j, 2019} + \frac{1}{2}\cdot avg(\beta_{i})$

donde $DELTA_{j, 2019}$ es el consumo el dia $j$ de 2019 y $\beta_{i}$ son los consumos durante las dos primeras semanas de Febrero de 2019 del contador $i$.

In [18]:
for i in tqdm(dicc_tipos['tipo_3']):
    predictions = []
    avg_beta = delta_df[i][delta_df.index <= '2019-02-14'].mean()
    for date in february_2019_two_weeks:
        delta_2019 = delta_df[i][date]
        predictions.append(0.5*delta_2019 + 0.5*avg_beta)
    results_df[i] = predictions

100%|████████████████████████████████████████████████████████████████████████████████| 44/44 [00:00<00:00, 1222.25it/s]


## Predicciones de contadores tipo 2<a name="id4"></a>

Hasta ahora hemos hecho predicciones groseras y naïves para apróximadamente un 7% de los contadores cuyos datos no eran buenos por un motivo u otro. Pero realmente el principal esfuerzo de predicción y donde vamos a emplear modelos de machine learning es con los contadores **tipo 2**, que recordemos son contadores que al menos tienen medidas para todo el mes de Enero de 2020.

Para estos contadores entrenaremos 2 modelos distintos:
- AutoARIMA
- XGBoost

Una vez tengamos las predicciones de cada uno de los dos modelos para las dos primeras semanas de Febrero, nuestra predicción final será la media de las predicciones de ambos modelos.

### AutoARIMA<a name="id41"></a>

El primer modelo que vamos a usar para los contadores de tipo 2 es un AutoARIMA. Este tipo de modelo está enfocado a series temporales de carácter estacional (como el consumo de agua) y se basa en hacer finetuning de los hiperparámetros (p,d,q) que habitualmente tiene un modelo ARIMA. Para la implementación hemos usado la biblioteca pmdarima.

https://alkaline-ml.com/pmdarima/modules/generated/pmdarima.arima.auto_arima.html

En la documentación arriba referenciada se puede consultar el significado de cada parámetro. Los más importantes que hemos seleccionado nosotros han sido:

- **m**: Número de periodos en cada "estación" que en este caso por tener daatos diarios y periodicidad semanal es 7 (consultar https://robjhyndman.com/hyndsight/seasonal-periods/).
- **stepwise**: Poniendo este parámetro a True el propio modelo se encarga de hacer finetuning sobre sí mismo dando pasos en la dirección de los parámetros que más mejora prometan.

En cada iteración del bucle de la siguiente celda tomaremos la serie temporal de un contador de tipo 2, entrenaremos un modelo AutoARIMA y haremos las correspondientes predicciones con el modelo ya entrenado. Guardaremos las predicciones en un dataframe *arima_results*.

In [403]:
arima_results = pd.DataFrame(columns = dicc_tipos['tipo_2'], index = prediction_dates)

for i in tqdm(dicc_tipos['tipo_2']):
    one_counter = delta_df[i][delta_df.index >= '2019-11-02'].dropna()
    
    arima_model = auto_arima(one_counter, 
                    start_p=0, 
                    d=1, 
                    start_q=0, 
                    max_p=3, 
                    max_d=3, 
                    max_q=3, 
                    start_P=0, 
                    D=1, 
                    start_Q=0, 
                    max_P=3, 
                    max_D=3, 
                    max_Q=3, 
                    m=7, 
                    seasonal=True, 
                    error_action='warn', 
                    trace=False, 
                    supress_warnings=True, 
                    stepwise=True, 
                    random_state=2517, 
                    #n_jobs=4,
                    n_fits=10)   
    arima_results[i] = arima_model.predict(n_periods = 14)

  0%|▏                                                                              | 5/2535 [00:38<5:18:37,  7.56s/it]C:\Users\sergi\Anaconda3\envs\cajamar\lib\site-packages\pmdarima\arima\auto.py:460: UserWarning: Input time-series is completely constant; returning a (0, 0, 0) ARMA.
  warnings.warn('Input time-series is completely constant; '
  4%|██▋                                                                           | 89/2535 [08:44<3:42:17,  5.45s/it]C:\Users\sergi\Anaconda3\envs\cajamar\lib\site-packages\pmdarima\arima\auto.py:460: UserWarning: Input time-series is completely constant; returning a (0, 0, 0) ARMA.
  warnings.warn('Input time-series is completely constant; '
  5%|███▌                                                                         | 117/2535 [11:28<4:12:54,  6.28s/it]C:\Users\sergi\Anaconda3\envs\cajamar\lib\site-packages\pmdarima\arima\_auto_solvers.py:522: ModelFitWarning: Error fitting  ARIMA(3,1,1)(3,1,1)[7]           (if you do not want to see th

 27%|████████████████████▏                                                      | 682/2535 [1:21:29<3:24:03,  6.61s/it]C:\Users\sergi\Anaconda3\envs\cajamar\lib\site-packages\pmdarima\arima\auto.py:460: UserWarning: Input time-series is completely constant; returning a (0, 0, 0) ARMA.
  warnings.warn('Input time-series is completely constant; '
 27%|████████████████████▏                                                      | 684/2535 [1:21:36<2:34:37,  5.01s/it]C:\Users\sergi\Anaconda3\envs\cajamar\lib\site-packages\pmdarima\arima\auto.py:460: UserWarning: Input time-series is completely constant; returning a (0, 0, 0) ARMA.
  warnings.warn('Input time-series is completely constant; '
 28%|████████████████████▊                                                      | 704/2535 [1:23:42<3:58:54,  7.83s/it]C:\Users\sergi\Anaconda3\envs\cajamar\lib\site-packages\pmdarima\arima\auto.py:460: UserWarning: Input time-series is completely constant; returning a (0, 0, 0) ARMA.
  warnings.warn('Inp

C:\Users\sergi\Anaconda3\envs\cajamar\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1899: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1
C:\Users\sergi\Anaconda3\envs\cajamar\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1906: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ma)**-1
 81%|█████████████████████████████████████████████████████████████▋              | 2059/2535 [3:55:41<26:48,  3.38s/it]C:\Users\sergi\Anaconda3\envs\cajamar\lib\site-packages\pmdarima\arima\auto.py:460: UserWarning: Input time-series is completely constant; returning a (0, 0, 0) ARMA.
  warnings.warn('Input time-series is completely constant; '
 85%|████████████████████████████████████████████████████████████████▌           | 2155/2535 [4:06:26<36:46,  5.81s/it]C:\Users\sergi\Anaconda3\envs\cajamar\lib\site-packages\pmdarima\arima\_auto_solvers.py:522: ModelFitWarning: Error fitt

### XGBoost<a name="id42"></a>

Como adelantábamos, el segundo modelo será XGBoost Regressor. Para enriquecer el modelo generaremos nuevas variables a partir de la variable fecha. Generaremos dos tipos de variables:
1. Temporales
2. Climáticas

Las **variables temporales** que generaremos serán las siguientes:
- Transformaciones seno y coseno del día de la semana. Es decir, a partir de la fecha obtenemos el día de la semana y a continuación hacemos las siguientes dos transformaciones para codificarlo como variables continuas ($W$ es el número del día de la semana siendo 1 Lunes, 2 Martes, etc.):

$sin_{W}=sin(\frac{2\cdot \pi \cdot W}{7}) \\
cos_{W}=cos(\frac{2\cdot \pi \cdot W}{7})$

- Transformaciones seno y coseno del día del año. De la misma manera que el día de la semana si $Y$ es el número del día del año (1 para el 1 de Enero, 2 para el 2 de Enero, etc.) hacemos las siguientes transormaciones:

$sin_{Y}=sin(\frac{2\cdot \pi \cdot Y}{365}) \\
cos_{Y}=cos(\frac{2\cdot \pi \cdot Y}{365})$

- Finalmente generamos una última variables *IS_WEEKEND* que tomará valor 1 si la fecha corresponde con un Sábado o un Domingo y 0 en caso contrario.

Por otra parte generamos las siguientes **variables climáticas** tomando los datos meteorológicos de una de las estaciones de AEMET en Valencia (https://opendata.aemet.es/centrodedescargas/inicio):
- Temperatura media
- Sol
- Precipitaciones

Una vez generadas todas estas variables (tanto para el conjunto de train como para las fechas de Febrero en las que tenemos que hacer la predicción) procedemos a iterar sobre cada contador del tipo 2 al igual que hicimos con los modelos AutoARIMA. Para cada contador $i$ entrenamos un XGBoost donde *X_train* está compuesto por las variables temporales y climáticas generadas en las fechas para las que $i$ tenga datos e *y_train* por los consumos del contador $i$. Una vez entrenado empleamos este modelo para predecir los consumos del contador $i$ de las dos primeras semanas de febrero.

In [121]:
def is_weekend(weekday):
    if weekday==5 or weekday==6:
        return 1
    else:
        return 0

start_date = datetime.date(2019, 11, 2)
end_date = datetime.date(2020, 2, 14)
all_dates = get_date_range(start_date, end_date)

time_variables = pd.DataFrame(index = all_dates)

# variables temporales
time_variables['DATE'] = all_dates
time_variables['DATE'] = time_variables['DATE'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
time_variables['WEEKDAY'] = time_variables['DATE'].apply(lambda x: x.weekday())
time_variables['YEAR_DAY'] = time_variables['DATE'].apply(lambda x: x.timetuple().tm_yday)
time_variables['IS_WEEKEND'] = time_variables['WEEKDAY'].apply(is_weekend)
days_in_a_week = 7
time_variables['sin_WEEKDAY'] = np.sin(2*np.pi*time_variables['WEEKDAY']/days_in_a_week)
time_variables['cos_WEEKDAY'] = np.cos(2*np.pi*time_variables['WEEKDAY']/days_in_a_week)
days_in_a_year = 365
time_variables['sin_year_day'] = np.sin(2*np.pi*time_variables['YEAR_DAY']/days_in_a_year)
time_variables['cos_year_day'] = np.cos(2*np.pi*time_variables['YEAR_DAY']/days_in_a_year)

time_variables.head()

,DATE,WEEKDAY,YEAR_DAY,IS_WEEKEND,sin_WEEKDAY,cos_WEEKDAY,sin_year_day,cos_year_day
2019-11-02,2019-11-02,5,306,1,-0.974928,-0.222521,-0.849817,0.527078
2019-11-03,2019-11-03,6,307,1,-0.781831,0.623490,-0.840618,0.541628
2019-11-04,2019-11-04,0,308,0,0.000000,1.000000,-0.831171,0.556017
2019-11-05,2019-11-05,1,309,0,0.781831,0.623490,-0.821477,0.570242
2019-11-06,2019-11-06,2,310,0,0.974928,-0.222521,-0.811539,0.584298


In [145]:
temperatura_valencia = pd.read_json('../data/Temperatura_Valencia.txt')
temperatura_valencia = temperatura_valencia[['fecha', 'tmed', 'tmin', 'tmax', 'sol', 'prec']]
temperatura_valencia['prec'] = temperatura_valencia['prec'].replace('Ip', None)
temperatura_valencia = temperatura_valencia.fillna(method='ffill')

temperatura_valencia['tmed'] = temperatura_valencia['tmed'].apply(lambda x: float(x.replace(',','.')))
temperatura_valencia['tmin'] = temperatura_valencia['tmin'].apply(lambda x: float(x.replace(',','.')))
temperatura_valencia['tmax'] = temperatura_valencia['tmax'].apply(lambda x: float(x.replace(',','.')))
temperatura_valencia['sol'] = temperatura_valencia['sol'].apply(lambda x: float(x.replace(',','.')))
temperatura_valencia['prec'] = temperatura_valencia['prec'].apply(lambda x: float(x.replace(',','.')))

In [146]:
temperatura_valencia[['tmed', 'tmin', 'tmax', 'sol', 'prec']].corr()

,tmed,tmin,tmax,sol,prec
tmed,1.000000,0.965356,0.959125,0.348302,-0.137641
tmin,0.965356,1.000000,0.852129,0.211119,-0.041143
tmax,0.959125,0.852129,1.000000,0.470573,-0.231800
sol,0.348302,0.211119,0.470573,1.000000,-0.405306
prec,-0.137641,-0.041143,-0.231800,-0.405306,1.000000


Como se podía esperar temperatura media, mínima y máxima están muy correladas por lo que nos quedamos solo con la media.

In [147]:
temperatura_valencia = temperatura_valencia[['fecha', 'tmed', 'sol', 'prec']]
temperatura_valencia['DATE'] = temperatura_valencia['fecha'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
temperatura_valencia.head()

,fecha,tmed,sol,prec,DATE
0,2019-01-01,11.9,6.0,0.0,2019-01-01
1,2019-01-02,12.0,8.7,0.0,2019-01-02
2,2019-01-03,10.5,7.7,0.0,2019-01-03
3,2019-01-04,10.2,8.4,0.0,2019-01-04
4,2019-01-05,12.4,8.7,0.0,2019-01-05


In [148]:
all_variables = pd.merge(time_variables, temperatura_valencia, on='DATE')
all_variables.drop(['WEEKDAY', 'YEAR_DAY'], axis=1, inplace=True)
all_variables.head()

,DATE,IS_WEEKEND,sin_WEEKDAY,cos_WEEKDAY,sin_year_day,cos_year_day,fecha,tmed,sol,prec
0,2019-11-02,1,-0.974928,-0.222521,-0.849817,0.527078,2019-11-02,24.8,5.7,0.0
1,2019-11-03,1,-0.781831,0.623490,-0.840618,0.541628,2019-11-03,22.8,7.6,0.0
2,2019-11-04,0,0.000000,1.000000,-0.831171,0.556017,2019-11-04,21.4,6.9,0.0
3,2019-11-05,0,0.781831,0.623490,-0.821477,0.570242,2019-11-05,18.2,9.2,0.0
4,2019-11-06,0,0.974928,-0.222521,-0.811539,0.584298,2019-11-06,17.6,9.1,0.0


In [171]:
xgboost_results = pd.DataFrame(columns = dicc_tipos['tipo_2'], index = prediction_dates)

for i in tqdm(dicc_tipos['tipo_2']):
    one_counter = delta_df[i][delta_df.index >= '2019-11-02'].dropna()
    one_counter_all_variables = pd.merge(one_counter, all_variables, left_on=one_counter.index, right_on='fecha', how='left')
    one_counter_all_variables['DELTA'] = one_counter_all_variables[i]
    one_counter_all_variables.drop(['DATE','fecha', i], axis=1, inplace=True)

    X_train = one_counter_all_variables.drop('DELTA', axis=1)
    y_train = one_counter_all_variables['DELTA']

    X_test = all_variables.drop(['DATE','fecha'], axis=1)[-14:]

    model = xgb.XGBRegressor(
        n_estimators=1000,
        reg_lambda=1,
        gamma=0,
        max_depth=5
    )
    model.fit(X_train, y_train)
    
    xgboost_results[i] =  model.predict(X_test)

100%|██████████████████████████████████████████████████████████████████████████████| 2534/2534 [19:45<00:00,  2.14it/s]


In [183]:
xgboost_results.to_pickle('../data/xgboost_results_1.pkl')

La predicción final que añadimos al dataframe *results_df* es la media entre la predicción del modelo AutoARIMA y la predicción del modelo XGBoost:

In [194]:
for i in tqdm(dicc_tipos['tipo_2']):
    results_df[i] = (arima_results[i] + xgboost_results[i])/2

100%|████████████████████████████████████████████████████████████████████████████| 2534/2534 [00:01<00:00, 1774.51it/s]


## Postprocesado<a name="id5"></a>

En este apartado detallamos los pequeños pasos de postprocesado antes de guardar la predicción final.

En primer lugar, dado que no tiene sentido predecir un consumo negativo, aquellos consumos predichos que sean negativos los ponemos a 0.

Finalmente pasamos nuestros resultados al formato requerido por el enunciado del concurso y guardamos el archivo. En este punto cabe resaltar que una de las pruebas que llevamos a cabo fue si era mejor emplear un modelo separado para calcular los consumos totales semanales. El resultado fue que era mejor predecir los consumos diarios y luego agregarlos por semanas.

In [211]:
results_df[results_df < 0] = 0
(results_df < 0).sum().sum()

0

In [232]:
final_results = results_df.T
final_results['Semana_1'] = final_results[['2020-02-01','2020-02-02','2020-02-03','2020-02-04','2020-02-05','2020-02-06','2020-02-07']].sum(axis=1)
final_results['Semana_2'] = final_results[['2020-02-08','2020-02-09','2020-02-10','2020-02-11','2020-02-12','2020-02-13','2020-02-14']].sum(axis=1)
final_results.drop(['2020-02-08','2020-02-09','2020-02-10','2020-02-11','2020-02-12','2020-02-13','2020-02-14'], axis=1, inplace=True)
final_results['ID'] = final_results.index
final_results = final_results[['ID','2020-02-01','2020-02-02','2020-02-03','2020-02-04','2020-02-05','2020-02-06','2020-02-07','Semana_1','Semana_2']]
final_results = round(final_results, 4)
final_results

,ID,2020-02-01,2020-02-02,2020-02-03,2020-02-04,2020-02-05,2020-02-06,2020-02-07,Semana_1,Semana_2
0,0,328.9435,311.2076,283.8836,293.8022,326.4230,296.3539,373.6840,2214.2978,2218.1058
1,1,4.1147,5.0981,10.4217,8.5452,5.5314,6.2061,5.1523,45.0694,40.6259
2,2,37.2041,39.9459,45.8988,46.3428,34.4310,30.7549,38.9017,273.4792,270.2077
3,3,463.4875,348.9227,294.2648,347.3875,380.6865,337.7756,429.2843,2601.8089,2639.6726
4,4,350.2188,335.4870,316.2345,309.3496,338.8879,336.3996,308.6230,2295.2004,2457.0977
...,...,...,...,...,...,...,...,...,...,...
2746,2746,649.0352,649.0352,649.0352,649.0352,649.0352,649.0352,649.0352,4543.2461,4543.2461
2747,2747,36.0000,36.0000,36.0000,36.0000,36.0000,36.0000,36.0000,252.0000,252.0000
2748,2748,442.5870,442.5870,442.5870,442.5870,442.5870,442.5870,442.5870,3098.1087,3098.1087
2749,2749,118.4536,118.4536,118.4536,118.4536,118.4536,118.4536,118.4536,829.1749,829.1749


In [233]:
path = '../data/La Manzana de Newton.txt'
final_results.to_csv(path, sep = '|', header=False, index=False)